<a href="https://colab.research.google.com/github/Ravindi24/DS_datatry/blob/main/CreditCard_FraudDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Dataset : Classification problem with under sampling**

Import files

In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Loading dataset

In [2]:
df = pd.read_csv('/content/creditcard.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0.0
1,0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0.0
2,1,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0.0
3,1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0.0
4,2,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0.0


In [4]:
#dataset information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49610 entries, 0 to 49609
Data columns (total 31 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Time    49610 non-null  int64  
 1   V1      49610 non-null  float64
 2   V2      49610 non-null  float64
 3   V3      49610 non-null  float64
 4   V4      49609 non-null  float64
 5   V5      49609 non-null  float64
 6   V6      49609 non-null  float64
 7   V7      49609 non-null  float64
 8   V8      49609 non-null  float64
 9   V9      49609 non-null  float64
 10  V10     49609 non-null  float64
 11  V11     49609 non-null  float64
 12  V12     49609 non-null  float64
 13  V13     49609 non-null  float64
 14  V14     49609 non-null  float64
 15  V15     49609 non-null  float64
 16  V16     49609 non-null  float64
 17  V17     49609 non-null  float64
 18  V18     49609 non-null  float64
 19  V19     49609 non-null  float64
 20  V20     49609 non-null  float64
 21  V21     49609 non-null  float64
 22

**Data Cleaning**

In [5]:
#missing values
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        1
V5        1
V6        1
V7        1
V8        1
V9        1
V10       1
V11       1
V12       1
V13       1
V14       1
V15       1
V16       1
V17       1
V18       1
V19       1
V20       1
V21       1
V22       1
V23       1
V24       1
V25       1
V26       1
V27       1
V28       1
Amount    1
Class     1
dtype: int64

In [6]:
df= df.dropna() #since there is a negligable amount of missing values, it is better to remove them
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
df['Class'].value_counts()  # This dataset is highly unbalanced

Class
0.0    49461
1.0      148
Name: count, dtype: int64

This dataset is highly unbalanced

0 --> Normal Transaction

1 --> Fraudulent Transaction

In [9]:
# seperating the data for analysis
legit = df[df.Class == 0]
fraud = df[df.Class == 1]

In [10]:
print(legit.shape)
print(fraud.shape)


(49461, 31)
(148, 31)


In [11]:
#statistical measure of the data
legit.Amount.describe()

count    49461.000000
mean        93.099593
std        253.325102
min          0.000000
25%          7.680000
50%         25.000000
75%         85.000000
max      12910.930000
Name: Amount, dtype: float64

In [12]:
fraud.Amount.describe()

count     148.000000
mean      100.170676
std       233.347471
min         0.000000
25%         1.000000
50%         9.560000
75%        99.990000
max      1809.680000
Name: Amount, dtype: float64

In [13]:
# compare values for both transactions
df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,28809.010473,-0.220239,-0.003947,0.726173,0.167967,-0.240785,0.111250,-0.096470,0.042413,0.134593,...,0.046113,-0.031134,-0.106800,-0.039391,0.008275,0.135623,0.020410,0.003001,0.004455,93.099593
1.0,26877.182432,-7.675017,5.448919,-10.398242,5.939649,-5.681176,-2.280722,-8.068966,3.739332,-3.587074,...,0.470319,0.886835,-0.225326,-0.284732,-0.084733,0.246743,0.155479,0.603171,0.030658,100.170676


**Under Sampling**

Build a sample dataset containing similar distribution of normal transactions and fraudlent transaction

No of fraudlent transaction=148

In [15]:
legit_sample=legit.sample(n=148)  # randomly assign entries to the sample

In [16]:
# concaatenate 2 dataframes and build new dataset
new_df = pd.concat([legit_sample,fraud],axis=0)

In [17]:
new_df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
37349,38917,-0.585219,0.665430,0.986950,-0.315865,0.862472,0.723537,0.549717,0.149285,-0.264199,...,-0.184783,-0.458951,-0.374725,-1.353367,0.114988,0.298725,-0.054401,0.088366,25.82,0.0
2583,2113,-0.737958,0.875208,1.430472,-2.004914,0.107402,-1.650284,1.095982,-0.400093,0.258693,...,0.177883,0.582424,-0.365181,0.765263,0.523743,-0.798359,-0.035274,0.106141,1.00,0.0
1704,1317,-0.391757,0.591094,2.752836,0.726359,0.359031,0.781081,0.994960,-0.862935,0.765892,...,-0.239918,0.195502,-0.463516,-0.410537,0.354929,-0.373488,-0.963554,-0.894061,36.19,0.0
3639,3110,1.116050,0.354819,-0.014956,2.154597,0.674664,0.711815,0.197260,0.062515,-0.746146,...,-0.217849,-0.793078,-0.111015,-1.364368,0.436061,-0.089671,-0.008447,0.020286,71.45,0.0
21268,31566,1.396071,-0.880736,-0.155852,-0.934196,-0.434641,0.526246,-0.811957,0.191336,-0.324428,...,-0.130671,-0.470917,-0.222418,-1.408007,0.605882,-0.223692,-0.002468,-0.007933,44.90,0.0


In [19]:
new_df['Class'].value_counts()

Class
0.0    148
1.0    148
Name: count, dtype: int64

In [18]:
new_df.groupby("Class").mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0.0,27421.500000,-0.216625,0.022681,0.874536,0.135020,-0.256111,0.120168,-0.122308,0.061436,0.314356,...,0.068895,0.017595,-0.078422,-0.067980,0.017562,0.212677,0.025284,0.012685,0.005616,85.943581
1.0,26877.182432,-7.675017,5.448919,-10.398242,5.939649,-5.681176,-2.280722,-8.068966,3.739332,-3.587074,...,0.470319,0.886835,-0.225326,-0.284732,-0.084733,0.246743,0.155479,0.603171,0.030658,100.170676


Splitting the data into features and classes

In [20]:
x =new_df.drop(columns='Class',axis=1)
y=new_df['Class']

In [21]:
x

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
37349,38917,-0.585219,0.665430,0.986950,-0.315865,0.862472,0.723537,0.549717,0.149285,-0.264199,...,0.005968,-0.184783,-0.458951,-0.374725,-1.353367,0.114988,0.298725,-0.054401,0.088366,25.82
2583,2113,-0.737958,0.875208,1.430472,-2.004914,0.107402,-1.650284,1.095982,-0.400093,0.258693,...,-0.072187,0.177883,0.582424,-0.365181,0.765263,0.523743,-0.798359,-0.035274,0.106141,1.00
1704,1317,-0.391757,0.591094,2.752836,0.726359,0.359031,0.781081,0.994960,-0.862935,0.765892,...,0.340517,-0.239918,0.195502,-0.463516,-0.410537,0.354929,-0.373488,-0.963554,-0.894061,36.19
3639,3110,1.116050,0.354819,-0.014956,2.154597,0.674664,0.711815,0.197260,0.062515,-0.746146,...,0.016344,-0.217849,-0.793078,-0.111015,-1.364368,0.436061,-0.089671,-0.008447,0.020286,71.45
21268,31566,1.396071,-0.880736,-0.155852,-0.934196,-0.434641,0.526246,-0.811957,0.191336,-0.324428,...,0.096491,-0.130671,-0.470917,-0.222418,-1.408007,0.605882,-0.223692,-0.002468,-0.007933,44.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46909,42985,-4.075975,0.963031,-5.076070,4.955963,-0.161437,-2.832663,-7.619765,1.618895,-2.992092,...,0.058861,1.030738,0.165328,-1.017502,-0.477983,-0.304987,-0.106089,1.899714,0.511462,1.00
46918,42988,-4.423508,1.648048,-6.934388,4.894601,-5.078131,0.010849,-3.409096,1.409291,-3.260672,...,-0.562264,0.698359,0.487478,1.228698,-0.535217,0.388278,-0.009466,2.300164,0.081231,648.00
46998,43028,-1.109646,0.811069,-1.138135,0.935265,-2.330248,-0.116106,-1.621986,0.458028,-0.912189,...,0.168891,0.641594,0.841755,0.176728,0.081004,-0.258899,0.707654,0.418649,0.080756,204.27
47802,43369,-3.365319,2.426503,-3.752227,0.276017,-2.305870,-1.961578,-3.029283,-1.674462,0.183961,...,-0.036837,2.070008,-0.512626,-0.248502,0.126550,0.104166,-1.055997,-1.200165,-1.012066,88.00


In [22]:
y

37349    0.0
2583     0.0
1704     0.0
3639     0.0
21268    0.0
        ... 
46909    1.0
46918    1.0
46998    1.0
47802    1.0
48094    1.0
Name: Class, Length: 296, dtype: float64

Split data into training and testing

In [23]:
x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [24]:
print(x.shape,x_train.shape,x_test.shape)

(296, 30) (236, 30) (60, 30)


Model training

**1) Logistic regression**

In [25]:
model1 = LogisticRegression()

In [26]:
model1.fit(x_train,y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

model evaluation

In [28]:
#accuracy on training data
x_train_pred1 = model1.predict(x_train)
train_accuracy1 = accuracy_score(x_train_pred1,y_train)
print("Accuracy on Training data : ", train_accuracy1)

Accuracy on Training data :  0.9915254237288136


In [29]:
#accuracy on test data
x_test_pred1 = model1.predict(x_test)
test_accuracy1 = accuracy_score(x_test_pred1,y_test)
print("Accuracy on Test data : ", test_accuracy1)

Accuracy on Test data :  0.9666666666666667


Parameter tunning - logistic reg

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'penalty': ['l1', 'l2','elasticnet'],    # l1=Lasso, l2=ridge
    'C': [0.001, 0.01, 0.1, 1, 10],          # inverse of regularization strength. Smaller values specify stronger regularization
    'solver': ['liblinear', 'saga'] ,         # used to optimize the logistic regression function
    'max_iter': [150,200, 250]               # maximum number of iterations taken for the solver to converge
}
grid_search = GridSearchCV(estimator=LogisticRegression(solver='lbfgs'), param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(x_train, y_train)
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

In [43]:
print("Best Parameters:", best_params)

Best Parameters: {'C': 0.1, 'max_iter': 150, 'penalty': 'l2', 'solver': 'liblinear'}


through hyper-parameter tuning, it gives Ridge as optimal

In [44]:
best_model = grid_search.best_estimator_
y_pred1 = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred1)
print("Accuracy:", accuracy)

Accuracy: 0.9666666666666667


**2) Random forest classifier**

In [45]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion="gini",
                             max_depth=8,
                             min_samples_split=10,
                             random_state=5)
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=8, min_samples_split=10, random_state=5)

In [46]:
y_pred2 = rf.predict(x_test)
y_pred2

array([0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1.,
       0., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
       0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 0., 1., 0., 0., 1.])

In [47]:
# overall accuracy of the model
accuracy_score(y_test,y_pred2)

0.9666666666666667

Hyper parameter tuning -random forest

In [50]:
param_grid = {
    'n_estimators': [75,150,200],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'min_samples_split':[2,5,8],
    'max_leaf_nodes': [3, 6, 9],
}
grid_search = GridSearchCV(RandomForestClassifier(),
                           param_grid=param_grid)
grid_search.fit(x_train, y_train)
print(grid_search.best_estimator_)

RandomForestClassifier(max_depth=9, max_features='log2', max_leaf_nodes=6,
                       n_estimators=75)


In [51]:
best_model = grid_search.best_estimator_
y_pred22 = best_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred22)
print("Accuracy:", accuracy)

Accuracy: 0.9833333333333333


3) XGBoost Classifier

In [59]:
import xgboost as xgb
xgb_model=xgb.XGBClassifier(objective="binary:logistic",random_state=21)
xgb_model.fit(x_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=21, ...)

In [60]:
y_pred3=xgb_model.predict(x_test)
accuracy_score(y_test,y_pred3)

1.0

parameter tuning- XGB

In [61]:
import scipy.stats as stats
from sklearn.model_selection import RandomizedSearchCV
param_dist = {
    'max_depth': stats.randint(3, 10),
    'learning_rate': stats.uniform(0.01, 0.1),
    'subsample': stats.uniform(0.5, 0.5),
    'n_estimators':stats.randint(50, 200)
}
xgb_model = xgb.XGBClassifier()
# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(xgb_model, param_distributions=param_dist, n_iter=10, cv=5, scoring='accuracy')

# Fit the RandomizedSearchCV object to the training data
xgb_optimal=random_search.fit(x_train, y_train)

# Print the best set of hyperparameters and the corresponding score
print("Best set of hyperparameters: ", random_search.best_params_)
print("Best score: ", random_search.best_score_)

Best set of hyperparameters:  {'learning_rate': 0.0946587247616397, 'max_depth': 5, 'n_estimators': 75, 'subsample': 0.7251812165710905}
Best score:  0.9618794326241135


In [62]:
best_model = random_search.best_params_
y_pred33=xgb_optimal.predict(x_test)
accuracy = accuracy_score(y_test, y_pred33)
print("Accuracy:", accuracy)

Accuracy: 0.9833333333333333


**XGBoost gives the best model for Credit Card Detection**